# Apriori

Here we discuss an implementation of an association rule learner due to Sebastian Raschka, author of the book *Python Machine Learning* (2e). It is part of his `mlxtend` package. 

*  https://rasbt.github.io/mlxtend/
*  https://github.com/rasbt/mlxtend

The bits for creating association rules are found in `mlxtend.frequent_patterns`. They are `apriori`, which creates item sets, and `association_rules`, which creates rules using confidence, lift, etc. 

The examples below are based on those the `mlxtend` site. 

### `apriori`

#### Usage

```python
apriori(df, min_support=0.5, use_colnames=False, max_len=None, n_jobs=1)
```

* `df` : pandas DataFrame or pandas SparseDataFrame
* `min_support` : float (default: 0.5). A float between 0 and 1 for minumum support of the itemsets returned. The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.
* `use_colnames` : bool (default: False). If true, uses the DataFrames' column names.
* `max_len` : int (default: None) Maximum length of the itemsets generated. If None (default), then all possible itemsets are evaluated.

Returns a pandas DataFrame with columns `['support', 'itemsets']`. Each item set is of type `frozenset`, an immutable set.

#### Example

Given a list of item sets, we can convert it to a boolean array using `TransactionEncoder`. This is then converted to a pandas `DataFrame`.

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

encoder = TransactionEncoder()
te_ary = encoder.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=encoder.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


Given a suitably encoded dataframe, we can use `apriori` to enumerate item-sets meeting a support threshold.

In [3]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


We can also specify that column names should be presented. 

In [4]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Kidney Beans, Yogurt)"


Once we have the item sets, we can record their sizes or other useful properties. 

In [5]:
df2 = apriori(df, min_support=0.6, use_colnames=True)
lengths = df2['itemsets'].apply(lambda x: len(x))
df2['size'] = lengths
df2

,support,itemsets,size
0,0.8,(Eggs),1
1,1.0,(Kidney Beans),1
2,0.6,(Milk),1
3,0.6,(Onion),1
4,0.6,(Yogurt),1
5,0.8,"(Kidney Beans, Eggs)",2
6,0.6,"(Onion, Eggs)",2
7,0.6,"(Milk, Kidney Beans)",2
8,0.6,"(Onion, Kidney Beans)",2
9,0.6,"(Kidney Beans, Yogurt)",2


In [6]:
df2[df2['size'] > 1].sort_values('support',ascending=False)

,support,itemsets,size
5,0.8,"(Kidney Beans, Eggs)",2
6,0.6,"(Onion, Eggs)",2
7,0.6,"(Milk, Kidney Beans)",2
8,0.6,"(Onion, Kidney Beans)",2
9,0.6,"(Kidney Beans, Yogurt)",2
10,0.6,"(Onion, Kidney Beans, Eggs)",3


###  `association_rules`

See: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/


```python
association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)
```
Returns a DataFrame of rules together with useful metrics such as `confidence`. 


* `df` : pandas DataFrame
* `metric` : string (default: 'confidence'). Automatically set to 'support' if support_only=True. Otherwise, supported metrics are 'support', 'confidence', 'lift', 'leverage', and 'conviction'.
* `min_threshold` : float (default: 0.8)
* `support_only` : bool (default: False). Only computes the rule support and fills the other metric columns with NaNs.

##### Support 
$support(A\rightarrow C)$ = $support(A \wedge C)$ range: [0, 1]

Support is defined primarily for itemsets and only indirectly for rules. Support really just indicates the frequency of a transaction containing all items of $A$ and $C$. 

##### Confidence 

$confidence(A\rightarrow C)$ = $\frac{support(A\wedge C)}{support(A)}$, range: [0, 1]

Confidence is really just the conditional probability of $C$ given $A$. That is $P(C|A) = \frac{P(A\wedge C)}{P(A)}$.
##### Lift 


$lift(A\rightarrow C)$ = $\frac{confidence(A\rightarrow C)}{support(C)}$, range: [0, $\infty$]

Lift quantifies how much better the rule is than random (i.e., how strongly $C$ is associated with $A$ in comparsion to being independent).  Given that confidence is just conditional probability, lift can be defined as: $\frac{P(C|A)}{P(C)}$. 
*  If $C$ and $A$ are independent of each other, then $P(C|A)$ is just $P(C)$, and so $lift(A\rightarrow C)$ = 1. 
*  If $A$ and $C$ are disjoint, then $lift(A\rightarrow C)$ = 0.
*  If $P(C|A)$ is large but $P(C)$ is small, then $lift(A\rightarrow C)$  will be large.

##### Leverage 

$leverage(A\rightarrow C)$ = $support(A \rightarrow C) - support(A)*support(C)$, range: [-1, 1]

The leverage of a rule compares the probility of $A$ and $C$ occurring together to the probability that they would co-occur if they were indenpendent of each other. If $A$ and $C$ are independent of each other then $P(AC)$ will be $P(A)P(C)$, and so the leverage will be 0.

##### Conviction 

$conviction(A\rightarrow C)$  = $\frac{[1 - support(C)]}{[1 - confidence(A\rightarrow C)]}$,range: [0, inf]

Again treating support and confidence as as probabilities, we get 

$conviction(A\rightarrow C)$  = $\frac{[1 - P(C)]}{[1 - P(C|A)]}$ = $\frac{P(\neg C)}{P(\neg C|A)}$ = $\frac{P(\neg C)}{P(\neg C|A)}$ = $\frac{P(\neg C)P(A)}{P(A \wedge\neg C)}$.

Brin et al. proposed Conviction handle cases like the following: If people buy milk ($M$) 90% of the time at a market and this is independent of whether they buy salmon ($S$),  then $confidence(S\rightarrow M)$ = 90%, which means a useless rule could be generated. 

As can be seen, if $A$ and $C$ are independent, then $conviction(A\rightarrow C)$  = 1.  However, suppose that $C$ occurs roughly half the time but that the conditional probability of $C$ given $A$ is very high (99%). Then $conviction(A\rightarrow C)$ will be much higher than 1. 


*  Sergey Brin, Rajeev Motwani, Jeffrey D. Ullman, and Shalom Turk. Dynamic itemset counting and implication rules for market basket data. In SIGMOD 1997, Proceedings ACM SIGMOD International Conference on Management of Data, pages 255-264, Tucson, Arizona, USA, May 1997



In [7]:
from mlxtend.frequent_patterns import association_rules

association_rules(df2, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
2,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
3,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
8,"(Onion, Eggs)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
9,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6


### Aside: From Test 2

In [8]:
# if all the instances are positive, then a random
# binary classifier with a cutoff of 0.25 should yield
# accuracy of 75%
import numpy as np
instance_count = 1000000 # use a lot of instances
pred = np.random.rand(instance_count)
cutoff = 0.25
# since all instances are positive, just count pred >= 0.25
pos_pred = np.sum(pred >= 0.25)

pos_pred/instance_count

0.750078